In [13]:
!pip install openai numpy pandas singlestoredb langchain==0.1.8 langchain-community==0.0.21 langchain-core==0.1.25 langchain-openai==0.0.6

  Using cached langchain_openai-0.0.6-py3-none-any.whl.metadata (2.5 kB)
Using cached langchain_openai-0.0.6-py3-none-any.whl (29 kB)
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.0.5
    Uninstalling langchain-openai-0.0.5:
      Successfully uninstalled langchain-openai-0.0.5


In [27]:
#Vector embeddings example
def word_to_vector(word):
    # Define some basic rules for our vector components
    vector = [0] * 5  # Initialize a vector of 5 dimensions

    # Rule 1: Length of the word (normalized to a max of 10 characters for simplicity)
    vector[0] = len(word) / 10

    # Rule 2: Number of vowels in the word (normalized to the length of the word)
    vowels = 'aeiou'
    vector[1] = sum(1 for char in word if char in vowels) / len(word)

    # Rule 3: Whether the word starts with a vowel (1) or not (0)
    vector[2] = 1 if word[0] in vowels else 0

    # Rule 4: Whether the word ends with a vowel (1) or not (0)
    vector[3] = 1 if word[-1] in vowels else 0

    # Rule 5: Percentage of consonants in the word
    vector[4] = sum(1 for char in word if char not in vowels and char.isalpha()) / len(word)

    return vector

# Example usage
word = "example"
vector = word_to_vector(word)
print(f"Word: {word}\nVector: {vector}")

Word: example
Vector: [0.7, 0.42857142857142855, 1, 1, 0.5714285714285714]


In [28]:
#Vector similarity example
import numpy as np

def cosine_similarity(vector_a, vector_b):
    # Calculate the dot product of vectors
    dot_product = np.dot(vector_a, vector_b)
    # Calculate the norm (magnitude) of each vector
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    # Calculate cosine similarity
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Example usage
word1 = "example"
word2 = "sample"
vector1 = word_to_vector(word1)
vector2 = word_to_vector(word2)

# Calculate and print cosine similarity
similarity_score = cosine_similarity(vector1, vector2)
print(f"Cosine similarity between '{word1}' and '{word2}': {similarity_score}")

Cosine similarity between 'example' and 'sample': 0.8108320947993962


In [10]:
import openai

# API anahtarınızı ayarlıyoruz
openai.api_key = ""

# Embedding talebini yapıyoruz
response = openai.embeddings.create(
    input="Golden Retriever",
    model="text-embedding-ada-002"  # Bu model adı, OpenAI'nin sağladığı embedding modeli
)

# Yanıtı yazdırıyoruz
print(response['data'][0]['embedding'])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [18]:
import os
os.environ['OPENAI_API_KEY'] = ‘mention your openai api key’

SyntaxError: invalid character '‘' (U+2018) (401536124.py, line 2)

In [19]:
#Creating a vector database with SingleStoreDB

import openai
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.singlestoredb import SingleStoreDB
import os
import pandas as pd
import requests

# URL of the public .txt file you want to use
file_url = "https://sherlock-holm.es/stories/plain-text/stud.txt"

# Send a GET request to the file URL
response = requests.get(file_url)

# Proceed if the file was successfully downloaded
if response.status_code == 200:
    file_content = response.text
    
    # Save the content to a file
    file_path = 'downloaded_example.txt'
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(file_content)
    
    # Now, you can proceed with your original code using 'downloaded_example.txt'
    # Load and process documents
    loader = TextLoader(file_path)  # Use the downloaded document

    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # Generate embeddings and create a document search database
    OPENAI_KEY = "add your openai key"  # Replace with your OpenAI API key
    embeddings = OpenAIEmbeddings(api_key=OPENAI_KEY)

    # Create Vector Database
    vector_database = SingleStoreDB.from_documents(docs, embeddings, table_name="scarlet")  # Replace "your_table_name" with your actual table name

    query = "which university did he study?"
    docs = vector_database.similarity_search(query)
    print(docs[0].page_content)

else:
    print("Failed to download the file. Please check the URL and try again.")

ConnectionError: HTTPSConnectionPool(host='sherlock-holm.es', port=443): Max retries exceeded with url: /stories/plain-text/stud.txt (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7eb218ad7990>: Failed to establish a new connection: [Errno 113] No route to host'))